# Setup

In [1]:
# import libraries
import numpy as np
from numpy import *
from bokeh.io import output_notebook, show
from bokeh.plotting import figure, ColumnDataSource
from bokeh.models import CustomJS, Slider, Range1d, Span, Toggle, Spinner
from bokeh.layouts import column, row, gridplot
output_notebook()

Loading BokehJS ...

In [2]:
# style definitions
colors = ['#000000', '#E69F00', '#56B4E9', '#009E73', '#F0E442', '#0072B2', '#D55E00', '#CC79A7']
SUB = str.maketrans("0123456789", "₀₁₂₃₄₅₆₇₈₉")
SUP = str.maketrans("0123456789", "⁰¹²³⁴⁵⁶⁷⁸⁹")

def set_fig_style(fig,xmax,ymax,xlab="x",ylab="y(x,t)",xmin=0,ymin=None):
    if ymin is None:
        ymin = -ymax
    for (ax, lab) in [(fig.xaxis,xlab),(fig.yaxis,ylab)]:
        ax.axis_label = lab
        ax.axis_label_text_font_size = '14pt'
        ax.axis_label_text_font_style = 'normal'
        ax.major_label_text_font_size = '12pt'
    fig.x_range = Range1d(xmin,xmax)
    fig.y_range = Range1d(ymin,ymax)

In [3]:
%%javascript
requirejs.config({
    paths: { 'math': ['https://cdnjs.cloudflare.com/ajax/libs/mathjs/11.6.0/math.min'], },
});

<IPython.core.display.Javascript object>

In [4]:
def make_string_animation(L,cs,v=1.,dt=None, width=900):
    # these are the coefficients of sin(kn x), with kn = nπ/L
    cs = np.asarray(cs)
    acs = abs(cs)
    phis = np.angle(cs)
    ns = np.arange(1,len(cs)+1)
    ks = ns*np.pi/L
    ωs = v*ks

    n = len(cs)
    # make sure that number of points in FFT is multiple of 2
    n_fft = 2**math.ceil(math.log2(2*n+2))
    cfft = np.zeros(n_fft)
    cfft[1:n+1] = n_fft*cs.real

    x = np.linspace(0,L,n_fft//2 + 1)

    if dt is None:
        T = 2*L/v
        dt = T/200

    if isinstance(dt,(int,float)):
        dt = (dt/10,dt*10,dt/10,dt)
    else:
        assert len(dt) == 4
    
    y = np.fft.ifft(cfft)[:len(x)]
    source = ColumnDataSource(data=dict(x=x,y=y.imag))

    fig = figure(width=width,height=500)
    fig.line('x', 'y', source=source, line_width=4, color=colors[0])
    set_fig_style(fig,x.max(),ymax=1.03*abs(source.data['y']).max())

    slider_t = Slider(start=0, end=1000, value=0, step=0.1, title='t', sizing_mode="stretch_width")
    spinner_dt = Spinner(low=dt[0], high=dt[1], step=dt[2], value=dt[3], title="δt", width=80, sizing_mode="stretch_height")

    # JavaScript callback to do animations directly in the browser (without needing to communicate with server)
    cb_sliders = CustomJS(args=dict(source=source, slider_t=slider_t, acs=acs, phis=phis, ωs=ωs, cfft=cfft),
                          code="""require(['math'], function(math) {
                                      const t = slider_t.value;
                                      const ys = source.data['y'];
                                      const n_fft = cfft.length;
                                      acs.forEach((ac,i) => cfft[i+1] = ac*math.cos(ωs[i]*t+phis[i]));
                                      const yfft = math.ifft(Array.from(cfft));
                                      ys.forEach((_,i) => ys[i] = n_fft*yfft[i].im);
                                      source.change.emit();
                                  });""")

    slider_t.js_on_change('value', cb_sliders)

    # Set up Play/Pause button/toggle JS
    cb_toggle = CustomJS(args=dict(s_t=slider_t,s_dt=spinner_dt,fig=fig),
                         code="""function upfun() { s_t.value += s_dt.value; cb_obj.active || clearInterval(updater) }
                                 cb_obj.label = cb_obj.active ? '❚❚ Pause' : '► Play';
                                 if (cb_obj.active) var updater = setInterval(upfun, 20);""")
    toggle = Toggle(label='► Play',active=False, sizing_mode="stretch_height", width=80)
    toggle.js_on_change('active', cb_toggle)

    layout = column(row(toggle, spinner_dt, slider_t, sizing_mode="stretch_width"), fig)
    show(layout)

# Plot motion of a string with fixed ends, with given normal mode coefficients

## Example 1
For initial condition $y(x,t=0) = a x (L-x)$, where $a$ is a constant with the correct units, we get $c_n = a \frac{4-4\cos(k_n L) - 2k_n L \sin(k_n L)}{k_n^3 L}$.

In [5]:
L = 1.
nmax = 150
n = np.arange(1,nmax+1)
kn = n*pi/L

# we use a=1
cn = (4 - 4*cos(kn*L) - 2*kn*L*sin(kn*L)) / (kn**3 * L)

make_string_animation(L,cn)

## Example 2: Problem 4.3
For the initial condition of problem 3 in problem sheet 4, we get $c_n = \frac{32 d}{3 n^2 \pi^2} \sin(n\pi/4)$. In the plot, we use $d=1$.

In [6]:
L = 2.
nmax = 500
n = np.arange(1,nmax+1)
kn = n*pi/L

cn = 32/(3*n**2*pi**2)*sin(n*pi/4)

make_string_animation(L,cn)

## Example 3: Problem 5.1
The below is ready to insert the expression for $c_n$ for problem 5.1, just below the line `##### insert your expression for c_n here #####`.

In [7]:
x1 = 0.95
x2 = 1.05

# do the plot both for L=2 and for L=3
for L in (2.,3.):
    # adjust nmax to get the same spatial resolution
    nmax = int(100*L)
    n = np.arange(1,nmax+1)
    kn = n*pi/L

    ##### insert your expression for c_n here #####
    cn =
    ###############################################

    make_string_animation(L,cn,width=int(300*L)+100,dt=0.01)

SyntaxError: invalid syntax (4177489038.py, line 12)